In [1]:
import pandas as pd
import time
import os
import openai
from dotenv import load_dotenv
from sklearn.metrics import classification_report

Initialize the OpenAI API by providing the API key, which is necessary to authenticate and make requests to GPT-3.5.


In [37]:
%%writefile .env
OPENAI_API_KEY=your-openai-api-key-here


Overwriting .env


In [32]:
!pip install python-dotenv


In [33]:

# Load the .env file
load_dotenv()

# Get API key from .env
openai.api_key = os.getenv("OPENAI_API_KEY")

if not openai.api_key:
    raise ValueError("Error: OpenAI API key not found! Make sure the .env file is correctly loaded.")


This function `get_gpt_sentiment()` sends the text to the OpenAI GPT-3.5 model, asking it to classify the sentiment as Positive or Negative.
The function handles errors gracefully and defaults to Negative sentiment in case of any exceptions.


In [34]:
def get_gpt_sentiment(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            request_timeout=10,
            messages=[
                {"role": "system", "content": "Classify this text strictly as Positive or Negative only."},
                {"role": "user", "content": text}
            ]
        )
        sentiment = response['choices'][0]['message']['content'].strip().lower()
        return "Positive" if "positive" in sentiment else "Negative"
    except Exception as e:
        print(f"Error: {e}")
        return "Negative"


We load the dataset and apply the GPT sentiment analysis function to the feedback texts. 
We also implement a sleep function to avoid hitting OpenAI's rate limit. 
Finally, we print a classification report to evaluate the performance of the GPT model.


In [35]:

# Load Data
base_dir = os.getcwd()
file_path = os.path.join(base_dir, "new_channel_partner_feedback.csv")
df = pd.read_csv(file_path)

# Analyze Sentiment for GPT (Add rate-limiting with sleep)
df["gpt_prediction"] = df["Feedback_Text"].apply(
    lambda x: (time.sleep(1), get_gpt_sentiment(x))[1]
)


# Print classification report
print("\n GPT Model Performance:")
print(classification_report(df["Sentiment"], df["gpt_prediction"]))



 GPT Model Performance:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00       500
    Positive       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [36]:
df

,Feedback_ID,Feedback_Text,Sentiment,gpt_prediction
0,4,"The product is well-engineered, leading to hig...",Positive,Positive
1,819,"Shipping partners are unreliable, causing delays.",Negative,Negative
2,621,Customer service representatives lack technica...,Negative,Negative
3,850,"The platform's navigation is unintuitive, maki...",Negative,Negative
4,201,Promotional discounts have driven more custome...,Positive,Positive
...,...,...,...,...
995,289,Customer demand has been steadily increasing f...,Positive,Positive
996,806,Support team is difficult to reach during crit...,Negative,Negative
997,721,Mobile notifications are excessive and cannot ...,Negative,Negative
998,57,The company maintains transparency in all tran...,Positive,Positive
